In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import json
import re

import numpy as np
import pandas as pd
import scipy.sparse as sp

In [5]:
%cd ../../

C:\Projects\python\recommender


In [6]:
DATA_BASE_PATH = './inputs/stackoverflow/'

## Generate Question Dataframe

In [11]:
questions = []

with open("./inputs/stackoverflow/question.json", mode='r', encoding='utf8') as fp:
    for line in fp:
        data = json.loads(line)
        questions.append({
            'votes': data['votes'],
            'answers': data['answers'],
            'views': data['views'],
            'questionId': data['questionId'],
            'tags': data['tags'],
            'time': data['time'],
            'userId': data['userId'],
            'userName': data['userName'],
            'reputation': data['reputation']
        })

len(questions)

610769

In [12]:
ques_df = pd.DataFrame(questions)
ques_df.head()

,answers,questionId,reputation,tags,time,userId,userName,views,votes
0,0,57986702,1,t-cñ t-windows t-user32,2019-09-18 06:55:05Z,12083331,alic,3 views,0
1,0,57986699,1,t-python t-graph,2019-09-18 06:54:54Z,12083319,fullmetal-alchem%c3%acst,4 views,0
2,0,57986698,37,t-google-bigquery t-tableau,2019-09-18 06:54:53Z,9203730,nikhil-boorla,3 views,0
3,0,57986696,558,t-javascript t-arrays,2019-09-18 06:54:45Z,1088949,kristina-bressler,15 views,0
4,0,57986692,31,t-javascript t-php t-jquery t-jquery-ui t-java...,2019-09-18 06:54:06Z,11977154,zain-shabir,11 views,0


In [13]:
ques_df.columns

Index(['answers', 'questionId', 'reputation', 'tags', 'time', 'userId',
       'userName', 'views', 'votes'],
      dtype='object')

In [14]:
ques_df = ques_df[[
    'questionId', 'answers', 'reputation', 'tags', 'views', 'time', 'userId',
    'userName', 'votes'
]]
ques_df.head()

,questionId,answers,reputation,tags,views,time,userId,userName,votes
0,57986702,0,1,t-cñ t-windows t-user32,3 views,2019-09-18 06:55:05Z,12083331,alic,0
1,57986699,0,1,t-python t-graph,4 views,2019-09-18 06:54:54Z,12083319,fullmetal-alchem%c3%acst,0
2,57986698,0,37,t-google-bigquery t-tableau,3 views,2019-09-18 06:54:53Z,9203730,nikhil-boorla,0
3,57986696,0,558,t-javascript t-arrays,15 views,2019-09-18 06:54:45Z,1088949,kristina-bressler,0
4,57986692,0,31,t-javascript t-php t-jquery t-jquery-ui t-java...,11 views,2019-09-18 06:54:06Z,11977154,zain-shabir,0


In [49]:
# ques_df['reputation'] = ques_df['reputation'].str.replace(',', '')
# ques_df['reputation'] = ques_df['reputation'].str.replace('k', '000')
# ques_df['reputation'] = ques_df['reputation'].str.replace('\.', '')
# ques_df['views'] = ques_df['views'].str[:-6]

In [53]:
# ques_df['questionId'] = ques_df['questionId'].astype('int32')
# ques_df['answers'] = ques_df['answers'].astype('int32')
# ques_df['reputation'] = ques_df['reputation'].astype('int32')
# ques_df['views'] = ques_df['views'].astype('int32')
ques_df['votes'] = ques_df['votes'].astype('int32')

In [54]:
ques_df.dtypes

questionId     int32
answers        int32
reputation     int32
tags          object
views          int32
time          object
userId        object
userName      object
votes          int32
dtype: object

In [55]:
ques_df.to_csv(DATA_BASE_PATH + 'question.csv', index=False)

## Generate Answer Dataframe

In [18]:
answers = []

with open('./inputs/stackoverflow/answer.json', mode='r', encoding='utf8') as fp:
    for line in fp:
        data = json.loads(line)
        answers.append({
            'questionId': data['queID'],
            'answerId': data['ansID'],
            'userId': data['userID'],
            'userName': data['userName'],
            'time': data['time'],
            'vote': data['vote'],
            'score': data['score']
        })
        
len(answers)

325247

In [19]:
for ans in answers:
    score = ans['score']
    if score[-1] == 'k':
        num = float(score[0:-1])
        num *= 1000
        ans['score'] = str(score)

In [30]:
s = '11.1k'
s = re.sub(r'k', '000', s)
s = re.sub(r'\.', '', s)
print(s)

111000


In [31]:
for ans in answers:
    score = ans['score']
    score = re.sub(r'k', '000', score)
    score = re.sub(r'\.', '', score)
    score = re.sub(r',', '', score)
    ans['score'] = score

In [32]:
ans_df = pd.DataFrame(answers)
ans_df.head()

,answerId,questionId,score,time,userId,userName,vote
0,57984385,57983805,661,2019-09-18 02:50:28Z,3148642,casper,0
1,57983913,57982932,159000,2019-09-18 01:40:10Z,463319,twisty,0
2,57983137,57982804,44,2019-09-17 23:22:56Z,1388032,korzhyk,0
3,57984007,57982804,8,2019-09-18 01:54:52Z,11237189,danial-dehvan,0
4,57982656,57982521,257000,2019-09-17 22:10:21Z,132438,felipe-hoffa,0


In [36]:
ans_df['score'] = ans_df['score'].astype('int32')

In [34]:
ans_df['vote'] = ans_df['vote'].astype('int32')

In [37]:
ans_df.dtypes

answerId      object
questionId    object
score          int32
time          object
userId        object
userName      object
vote           int32
dtype: object

In [38]:
ans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325247 entries, 0 to 325246
Data columns (total 7 columns):
answerId      325247 non-null object
questionId    325247 non-null object
score         325247 non-null int32
time          325247 non-null object
userId        325247 non-null object
userName      325247 non-null object
vote          325247 non-null int32
dtypes: int32(2), object(5)
memory usage: 14.9+ MB


In [39]:
ans_df.to_csv("./inputs/stackoverflow/answer.csv", index=False)

## Generate User Dataframe

In [73]:
user_df = ans_df[['userId', 'userName', 'score']]
user_df.head()

,userId,userName,score
0,3148642,casper,661
1,463319,twisty,159000
2,1388032,korzhyk,44
3,11237189,danial-dehvan,8
4,132438,felipe-hoffa,257000


In [74]:
user_df.shape

(325247, 3)

In [75]:
user_df = user_df.drop_duplicates(subset=['userId'])
user_df.shape

(96700, 3)

In [76]:
user_df.to_csv("./inputs/stackoverflow/user.csv", index=False)

### Drop user info in question dataframe

In [10]:
ques_df = ques_df.drop(columns=['userName', 'reputation'])
ques_df.head()

,questionId,answers,tags,views,time,userId,votes
0,57986702,0,t-cñ t-windows t-user32,3,2019-09-18 06:55:05Z,12083331,0
1,57986699,0,t-python t-graph,4,2019-09-18 06:54:54Z,12083319,0
2,57986698,0,t-google-bigquery t-tableau,3,2019-09-18 06:54:53Z,9203730,0
3,57986696,0,t-javascript t-arrays,15,2019-09-18 06:54:45Z,1088949,0
4,57986692,0,t-javascript t-php t-jquery t-jquery-ui t-java...,11,2019-09-18 06:54:06Z,11977154,0


In [12]:
ques_df = ques_df[['questionId',  'userId', 'answers', 'tags', 'views', 'time', 'votes']]
ques_df.head()

,questionId,userId,answers,tags,views,time,votes
0,57986702,12083331,0,t-cñ t-windows t-user32,3,2019-09-18 06:55:05Z,0
1,57986699,12083319,0,t-python t-graph,4,2019-09-18 06:54:54Z,0
2,57986698,9203730,0,t-google-bigquery t-tableau,3,2019-09-18 06:54:53Z,0
3,57986696,1088949,0,t-javascript t-arrays,15,2019-09-18 06:54:45Z,0
4,57986692,11977154,0,t-javascript t-php t-jquery t-jquery-ui t-java...,11,2019-09-18 06:54:06Z,0


In [14]:
ques_df = ques_df.rename(columns={'userId': 'askUserId'})
ques_df.head()

,questionId,askUserId,answers,tags,views,time,votes
0,57986702,12083331,0,t-cñ t-windows t-user32,3,2019-09-18 06:55:05Z,0
1,57986699,12083319,0,t-python t-graph,4,2019-09-18 06:54:54Z,0
2,57986698,9203730,0,t-google-bigquery t-tableau,3,2019-09-18 06:54:53Z,0
3,57986696,1088949,0,t-javascript t-arrays,15,2019-09-18 06:54:45Z,0
4,57986692,11977154,0,t-javascript t-php t-jquery t-jquery-ui t-java...,11,2019-09-18 06:54:06Z,0


In [36]:
ques_df = ques_df.rename(columns={'votes': 'questionVote'})
ques_df.head()

,questionId,askUserId,answers,tags,views,time,questionVote
0,57986702,12083331,0,t-cñ t-windows t-user32,3,2019-09-18 06:55:05Z,0
1,57986699,12083319,0,t-python t-graph,4,2019-09-18 06:54:54Z,0
2,57986698,9203730,0,t-google-bigquery t-tableau,3,2019-09-18 06:54:53Z,0
3,57986696,1088949,0,t-javascript t-arrays,15,2019-09-18 06:54:45Z,0
4,57986692,11977154,0,t-javascript t-php t-jquery t-jquery-ui t-java...,11,2019-09-18 06:54:06Z,0


In [37]:
# Save dataframe
ques_df.to_csv("./inputs/stackoverflow/question.csv", index=False)

### Drop user information in answer dataframe

In [22]:
ans_df.head()

,questionId,score,time,userId,userName,vote
0,57983805,661,2019-09-18 02:50:28Z,3148642,casper,0
1,57982932,159000,2019-09-18 01:40:10Z,463319,twisty,0
2,57982804,44,2019-09-17 23:22:56Z,1388032,korzhyk,0
3,57982804,8,2019-09-18 01:54:52Z,11237189,danial-dehvan,0
4,57982521,257000,2019-09-17 22:10:21Z,132438,felipe-hoffa,0


In [23]:
ans_df = ans_df.drop(columns=['score', 'userName'])
ans_df.head()

,questionId,time,userId,vote
0,57983805,2019-09-18 02:50:28Z,3148642,0
1,57982932,2019-09-18 01:40:10Z,463319,0
2,57982804,2019-09-17 23:22:56Z,1388032,0
3,57982804,2019-09-18 01:54:52Z,11237189,0
4,57982521,2019-09-17 22:10:21Z,132438,0


In [24]:
ans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325247 entries, 0 to 325246
Data columns (total 4 columns):
questionId    325247 non-null int64
time          325247 non-null object
userId        325247 non-null int64
vote          325247 non-null int64
dtypes: int64(3), object(1)
memory usage: 9.9+ MB


In [33]:
ans_df = ans_df.rename(columns={'vote': 'answerVote'})
ans_df.head()

,questionId,time,userId,answerVote
0,57983805,2019-09-18 02:50:28Z,3148642,0
1,57982932,2019-09-18 01:40:10Z,463319,0
2,57982804,2019-09-17 23:22:56Z,1388032,0
3,57982804,2019-09-18 01:54:52Z,11237189,0
4,57982521,2019-09-17 22:10:21Z,132438,0


In [38]:
ans_df.to_csv("./inputs/stackoverflow/answer.csv", index=False)

## Generate Item Dataframe

In [39]:
ques_df = pd.read_csv("./inputs/stackoverflow/question.csv")
ans_df = pd.read_csv("./inputs/stackoverflow/answer.csv")

In [40]:
ques_df.head()

,questionId,askUserId,answers,tags,views,time,questionVote
0,57986702,12083331,0,t-cñ t-windows t-user32,3,2019-09-18 06:55:05Z,0
1,57986699,12083319,0,t-python t-graph,4,2019-09-18 06:54:54Z,0
2,57986698,9203730,0,t-google-bigquery t-tableau,3,2019-09-18 06:54:53Z,0
3,57986696,1088949,0,t-javascript t-arrays,15,2019-09-18 06:54:45Z,0
4,57986692,11977154,0,t-javascript t-php t-jquery t-jquery-ui t-java...,11,2019-09-18 06:54:06Z,0


In [41]:
ques_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610769 entries, 0 to 610768
Data columns (total 7 columns):
questionId      610769 non-null int64
askUserId       610769 non-null int64
answers         610769 non-null int64
tags            610769 non-null object
views           610769 non-null int64
time            610769 non-null object
questionVote    610769 non-null int64
dtypes: int64(5), object(2)
memory usage: 32.6+ MB


In [42]:
ans_df.head()

,questionId,time,userId,answerVote
0,57983805,2019-09-18 02:50:28Z,3148642,0
1,57982932,2019-09-18 01:40:10Z,463319,0
2,57982804,2019-09-17 23:22:56Z,1388032,0
3,57982804,2019-09-18 01:54:52Z,11237189,0
4,57982521,2019-09-17 22:10:21Z,132438,0


In [43]:
ans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325247 entries, 0 to 325246
Data columns (total 4 columns):
questionId    325247 non-null int64
time          325247 non-null object
userId        325247 non-null int64
answerVote    325247 non-null int64
dtypes: int64(3), object(1)
memory usage: 9.9+ MB


In [44]:
item_df = pd.merge(
    left=ans_df,
    right=ques_df[['questionId', 'answers', 'tags', 'views', 'questionVote']],
    on='questionId',
    how='inner')
item_df.head()

,questionId,time,userId,answerVote,answers,tags,views,questionVote
0,57983805,2019-09-18 02:50:28Z,3148642,0,1,t-javascript t-arrays t-set t-theory,26,-4
1,57982932,2019-09-18 01:40:10Z,463319,0,1,t-jquery t-aspûnet t-ajax t-model-view-controller,16,0
2,57982804,2019-09-17 23:22:56Z,1388032,0,2,t-javascript t-nodeûjs t-express t-post t-get,32,0
3,57982804,2019-09-18 01:54:52Z,11237189,0,2,t-javascript t-nodeûjs t-express t-post t-get,32,0
4,57982521,2019-09-17 22:10:21Z,132438,0,1,t-google-bigquery,17,0


In [45]:
item_df.shape

(299845, 8)

In [47]:
item_df = item_df[[
    'questionId', 'userId', 'time', 'answerVote', 'answers', 'tags', 'views',
    'questionVote'
]]
item_df.head()

,questionId,userId,time,answerVote,answers,tags,views,questionVote
0,57983805,3148642,2019-09-18 02:50:28Z,0,1,t-javascript t-arrays t-set t-theory,26,-4
1,57982932,463319,2019-09-18 01:40:10Z,0,1,t-jquery t-aspûnet t-ajax t-model-view-controller,16,0
2,57982804,1388032,2019-09-17 23:22:56Z,0,2,t-javascript t-nodeûjs t-express t-post t-get,32,0
3,57982804,11237189,2019-09-18 01:54:52Z,0,2,t-javascript t-nodeûjs t-express t-post t-get,32,0
4,57982521,132438,2019-09-17 22:10:21Z,0,1,t-google-bigquery,17,0


In [48]:
item_df.to_csv("./inputs/stackoverflow/item.csv", index=False)

## Preprocess Item Dataframe

In [7]:
item_df = pd.read_csv("./inputs/stackoverflow/item.csv")
item_df.head()

,questionId,userId,time,answerVote,answers,tags,views,questionVote
0,57983805,3148642,2019-09-18 02:50:28Z,0,1,t-javascript t-arrays t-set t-theory,26,-4
1,57982932,463319,2019-09-18 01:40:10Z,0,1,t-jquery t-aspûnet t-ajax t-model-view-controller,16,0
2,57982804,1388032,2019-09-17 23:22:56Z,0,2,t-javascript t-nodeûjs t-express t-post t-get,32,0
3,57982804,11237189,2019-09-18 01:54:52Z,0,2,t-javascript t-nodeûjs t-express t-post t-get,32,0
4,57982521,132438,2019-09-17 22:10:21Z,0,1,t-google-bigquery,17,0


In [8]:
item_df['time'] = pd.to_datetime(item_df['time'], infer_datetime_format=True)
item_df.dtypes

questionId               int64
userId                   int64
time            datetime64[ns]
answerVote               int64
answers                  int64
tags                    object
views                    int64
questionVote             int64
dtype: object

In [9]:
item_df.isna().any()

questionId      False
userId          False
time            False
answerVote      False
answers         False
tags            False
views           False
questionVote    False
dtype: bool

In [10]:
print(f"Raw dataframe shape: {item_df.shape}")

Raw dataframe shape: (299845, 8)


In [11]:
user_counts = item_df['userId'].value_counts()
user_counts.head()

1144035    1843
2901002     778
3732271     774
3962914     688
67579       596
Name: userId, dtype: int64

In [12]:
print(f"Original user size: {user_counts.size}")

Original user size: 75572


In [13]:
user_counts = user_counts[user_counts >= 4]
print(f"Filtered user size: {user_counts.size}")

Filtered user size: 14795


In [14]:
item_df = item_df[item_df['userId'].isin(user_counts.index)]
print(f"Filtered item dataframe shape: {item_df.shape}")

Filtered item dataframe shape: (217223, 8)


In [15]:
item_df.head()

,questionId,userId,time,answerVote,answers,tags,views,questionVote
0,57983805,3148642,2019-09-18 02:50:28,0,1,t-javascript t-arrays t-set t-theory,26,-4
1,57982932,463319,2019-09-18 01:40:10,0,1,t-jquery t-aspûnet t-ajax t-model-view-controller,16,0
4,57982521,132438,2019-09-17 22:10:21,0,1,t-google-bigquery,17,0
6,57982535,2801913,2019-09-17 22:21:28,0,2,t-bash,22,1
8,57982539,2585865,2019-09-17 22:06:26,0,2,t-java t-android t-backup t-database-restore,24,0


### Generate question dataframe

In [16]:
ques_cols = ['questionId', 'tags', 'answers', 'views', 'questionVote']

In [17]:
ques_df = item_df[ques_cols]
ques_df.head()

,questionId,tags,answers,views,questionVote
0,57983805,t-javascript t-arrays t-set t-theory,1,26,-4
1,57982932,t-jquery t-aspûnet t-ajax t-model-view-controller,1,16,0
4,57982521,t-google-bigquery,1,17,0
6,57982535,t-bash,2,22,1
8,57982539,t-java t-android t-backup t-database-restore,2,24,0


In [18]:
print(f"Raw question dataframe shape: {ques_df.shape}")
ques_df = ques_df.drop_duplicates(subset=['questionId'])
print(f"Filtered question dataframe shape: {ques_df.shape}")

Raw question dataframe shape: (217223, 5)
Filtered question dataframe shape: (159371, 5)


In [19]:
ques_df.loc[-1] = [0, '', 0, 0, 0]
ques_df.loc[-1]

questionId      0
tags             
answers         0
views           0
questionVote    0
Name: -1, dtype: object

In [20]:
ques_df['tags'] = ques_df['tags'].str.split(' ')
ques_df.head()

,questionId,tags,answers,views,questionVote
0,57983805,"[t-javascript, t-arrays, t-set, t-theory]",1,26,-4
1,57982932,"[t-jquery, t-aspûnet, t-ajax, t-model-view-con...",1,16,0
4,57982521,[t-google-bigquery],1,17,0
6,57982535,[t-bash],2,22,1
8,57982539,"[t-java, t-android, t-backup, t-database-restore]",2,24,0


In [21]:
ques_df.loc[-1]

questionId       0
tags            []
answers          0
views            0
questionVote     0
Name: -1, dtype: object

In [22]:
ques_df.dtypes

questionId       int64
tags            object
answers          int64
views            int64
questionVote     int64
dtype: object

In [23]:
ques_df.isna().any()

questionId      False
tags            False
answers         False
views           False
questionVote    False
dtype: bool

#### Drop question information columns

In [24]:
item_df = item_df[['questionId', 'userId', 'time', 'answerVote']]
item_df.head()

,questionId,userId,time,answerVote
0,57983805,3148642,2019-09-18 02:50:28,0
1,57982932,463319,2019-09-18 01:40:10,0
4,57982521,132438,2019-09-17 22:10:21,0
6,57982535,2801913,2019-09-17 22:21:28,0
8,57982539,2585865,2019-09-17 22:06:26,0


In [25]:
item_df.isna().any()

questionId    False
userId        False
time          False
answerVote    False
dtype: bool

### Generate previous columns

In [26]:
prev_cols = ['prevQuesId', 'answerVote']

In [27]:
item_df['prevQuesId'] = item_df['questionId']
item_df.head()

,questionId,userId,time,answerVote,prevQuesId
0,57983805,3148642,2019-09-18 02:50:28,0,57983805
1,57982932,463319,2019-09-18 01:40:10,0,57982932
4,57982521,132438,2019-09-17 22:10:21,0,57982521
6,57982535,2801913,2019-09-17 22:21:28,0,57982535
8,57982539,2585865,2019-09-17 22:06:26,0,57982539


In [28]:
item_df = item_df.sort_values(by=['userId', 'time'])
item_df.head()

,questionId,userId,time,answerVote,prevQuesId
244989,57281293,188,2019-07-31 01:23:36,0,57281293
242860,57287038,188,2019-07-31 09:21:25,0,57287038
243945,57285932,188,2019-07-31 09:31:43,1,57285932
238254,57300817,188,2019-08-01 01:50:28,0,57300817
238306,57300846,188,2019-08-01 01:53:09,2,57300846


In [29]:
item_df[prev_cols] = item_df[prev_cols].shift(periods=1, fill_value=0)
item_df.head()

,questionId,userId,time,answerVote,prevQuesId
244989,57281293,188,2019-07-31 01:23:36,0,0
242860,57287038,188,2019-07-31 09:21:25,0,57281293
243945,57285932,188,2019-07-31 09:31:43,0,57287038
238254,57300817,188,2019-08-01 01:50:28,1,57285932
238306,57300846,188,2019-08-01 01:53:09,0,57300817


In [30]:
first_mask = item_df.duplicated(subset=['userId'], keep='first')
first_mask.head()

244989    False
242860     True
243945     True
238254     True
238306     True
dtype: bool

In [31]:
item_df[prev_cols] = item_df[prev_cols].where(first_mask, 0)
item_df[~first_mask].head()

,questionId,userId,time,answerVote,prevQuesId
244989,57281293,188,2019-07-31 01:23:36,0,0
240777,57295065,267,2019-07-31 16:54:16,0,0
214253,57367403,371,2019-08-06 00:40:40,0,0
291056,57155514,476,2019-07-23 01:29:31,0,0
298146,57133744,967,2019-07-22 09:21:06,0,0


In [32]:
item_df.head()

,questionId,userId,time,answerVote,prevQuesId
244989,57281293,188,2019-07-31 01:23:36,0,0
242860,57287038,188,2019-07-31 09:21:25,0,57281293
243945,57285932,188,2019-07-31 09:31:43,0,57287038
238254,57300817,188,2019-08-01 01:50:28,1,57285932
238306,57300846,188,2019-08-01 01:53:09,0,57300817


In [33]:
item_df[item_df['answerVote'].isna()]

,questionId,userId,time,answerVote,prevQuesId


In [34]:
item_df.dtypes

questionId             int64
userId                 int64
time          datetime64[ns]
answerVote             int64
prevQuesId             int64
dtype: object

In [35]:
item_df[prev_cols] = item_df[prev_cols].astype('int64')
item_df.dtypes

questionId             int64
userId                 int64
time          datetime64[ns]
answerVote             int64
prevQuesId             int64
dtype: object

In [36]:
ques_df.head()

,questionId,tags,answers,views,questionVote
0,57983805,"[t-javascript, t-arrays, t-set, t-theory]",1,26,-4
1,57982932,"[t-jquery, t-aspûnet, t-ajax, t-model-view-con...",1,16,0
4,57982521,[t-google-bigquery],1,17,0
6,57982535,[t-bash],2,22,1
8,57982539,"[t-java, t-android, t-backup, t-database-restore]",2,24,0


In [37]:
part_df = ques_df.iloc[0:2][['questionId']]
part_df.head()

,questionId
0,57983805
1,57982932


In [38]:
part_df.loc[2] = 57982932
part_df.head()

,questionId
0,57983805
1,57982932
2,57982932


In [39]:
pd.merge(left=part_df, right=ques_df, on='questionId', how='inner')

,questionId,tags,answers,views,questionVote
0,57983805,"[t-javascript, t-arrays, t-set, t-theory]",1,26,-4
1,57982932,"[t-jquery, t-aspûnet, t-ajax, t-model-view-con...",1,16,0
2,57982932,"[t-jquery, t-aspûnet, t-ajax, t-model-view-con...",1,16,0


### Add Encoder

In [64]:
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, MinMaxScaler

In [65]:
conts_name = ['answers', 'views', 'questionVote']

In [66]:
question_encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
user_encoder = OneHotEncoder(categories='auto', handle_unknown='error')
ans_vote_scaler = MinMaxScaler(feature_range=(0, 1))
conts_scaler = MinMaxScaler(feature_range=(0, 1))
tags_binzer = MultiLabelBinarizer(sparse_output=True)

In [67]:
question_encoder.fit(item_df[['questionId']])
user_encoder.fit(item_df[['userId']])
ans_vote_scaler.fit(item_df[['answerVote']])
conts_scaler.fit(ques_df[conts_name])
# tags_binzer.fit(ques_df[['tags']])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [68]:
tags_binzer.fit(ques_df['tags'])
tags_binzer

MultiLabelBinarizer(classes=None, sparse_output=True)

In [69]:
tags_binzer.classes_

array(['', 'pt', 't-12factor', ..., 't-ûpost', 't-ûso', 't-ûwhen'],
      dtype=object)

#### Generate sample

In [74]:
df = item_df.sample(n=100)

In [100]:
# Generate feature dataframe
pos_df: pd.DataFrame = pd.merge(left=df[['questionId']],
                                right=ques_df,
                                on='questionId',
                                how='inner')
prev_df = pd.merge(left=df[['prevQuesId']],
                   right=ques_df,
                   left_on='prevQuesId',
                   right_on='questionId',
                   how='inner')
neg_df = ques_df.sample(df.shape[0])

In [101]:
prev_df.head()

,prevQuesId,questionId,tags,answers,views,questionVote
0,57558600,57558600,[t-python],2,52,1
1,57576492,57576492,"[t-typescript, t-types, t-interface]",2,32,4
2,57786356,57786356,[t-java],1,30,0
3,57653488,57653488,"[t-r, t-dataframe]",2,67,2
4,57428261,57428261,"[t-python, t-numpy]",2,48,0


In [73]:
tags_mat = tags_binzer.transform(pos_df['tags'])
tags_mat.shape

(100, 18224)

In [102]:
user_mat: sp.csr_matrix = user_encoder.transform(df[['userId']])
prev_ques_mat = question_encoder.transform(df[['prevQuesId']])
pos_ques_mat = question_encoder.transform(df[['questionId']])
neg_ques_mat = question_encoder.transform(neg_df[['questionId']])
ans_vote_mat = ans_vote_scaler.transform(df[['answerVote']])
prev_tags_mat = tags_binzer.transform(prev_df['tags'])
pos_tags_mat = tags_binzer.transform(pos_df['tags'])
neg_tags_mat = tags_binzer.transform(neg_df['tags'])
prev_cont_mat = sp.csr_matrix(conts_scaler.transform(prev_df[conts_name]))
pos_cont_mat = sp.csr_matrix(conts_scaler.transform(pos_df[conts_name]))
neg_cont_mat = sp.csr_matrix(conts_scaler.transform(neg_df[conts_name]))

In [103]:
pos_mat = sp.hstack([
    user_mat, prev_ques_mat, pos_ques_mat, ans_vote_mat, prev_tags_mat,
    prev_cont_mat, pos_tags_mat, pos_cont_mat
])
neg_mat = sp.hstack([
    user_mat, prev_ques_mat, neg_ques_mat, ans_vote_mat, prev_tags_mat,
    prev_cont_mat, neg_tags_mat, neg_cont_mat
])

In [94]:
print(f"pos shape {pos_mat.shape}, neg shape: {neg_mat.shape}")

pos shape (100, 351765), neg shape: (100, 351765)


In [88]:
# Generate feature dimension
question_dim = question_encoder.categories_[0].size
user_dim = user_encoder.categories_[0].size
ans_vote_dim = 1
cont_dim = len(conts_name)
tags_dim = tags_binzer.classes_.size

In [90]:
print(question_dim)
print(pos_ques_mat.shape)

159371
(100, 159371)


In [97]:
ans_vote_mat.shape

(100, 1)

In [104]:
user_dim + 2 * question_dim + ans_vote_dim + 2 * cont_dim + 2 * tags_dim

369992

In [105]:
pos_mat.shape

(100, 369992)

In [83]:
print(f"user dim: {user_mat.shape}")
print(f"encoder dim: {user_encoder.categories_[0].size}")

user dim: (100, 14795)
encoder dim: 14795


In [84]:
print(f"question dim: {question_encoder.categories_[0].size}")
print(f"question matrix dim: {pos_ques_mat.shape}")
print(f"previous matrix dim: {prev_ques_mat.shape}")

question dim: 159371
question matrix dim: (100, 159371)
previous matrix dim: (100, 159371)
